In [1]:
from get_stock_data import Downloader, mkdir
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import catboost as cb

# 获取全部股票的日K线数据
mkdir('stockdata/d_data')
raw_train_path = 'stockdata/d_train'
raw_test_path = 'stockdata/d_test'
train_path = 'stockdata/d_data/train.csv'
test_path = 'stockdata/d_data/test.csv'
mode = 'debug'

In [2]:
if mode == 'debug':
    train = pd.read_csv(train_path, nrows=100000)
    test = pd.read_csv(test_path, nrows=100000)
else:
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)

In [3]:
train['code'].unique()

array(['sh.600358', 'sz.002233', 'sh.600956', 'sz.301136', 'sz.002582',
       'sh.600999', 'sh.605151', 'sh.688190', 'sz.000062', 'sz.300218',
       'sh.600864', 'sz.399997', 'sz.399903', 'sh.600811', 'sh.601599',
       'sz.300695', 'sz.003002', 'sz.399638', 'sh.601588', 'sz.002113',
       'sh.600640', 'sh.603600', 'sz.300739', 'sh.000004', 'sz.301163',
       'sz.003037', 'sh.603536', 'sh.603989', 'sh.600308', 'sh.603380',
       'sz.002790', 'sz.003031', 'sz.399316', 'sh.605288', 'sz.002532',
       'sh.603517', 'sh.000928', 'sh.688393'], dtype=object)

In [4]:
train = train[train['code']=='sz.301136']
test = test[test['code']=='sz.301136']

In [5]:
def feature_engineer(train, test):
    train_len = len(train)
    data = pd.concat((train, test), sort=False).reset_index(drop=True)
    
    stock_industry = pd.read_csv("stock_industry.csv", encoding="gbk")
    from sklearn.preprocessing import LabelEncoder
    lbe = LabelEncoder()
    stock_industry['industry'] = lbe.fit_transform(stock_industry['industry'])
    data = pd.merge(data, stock_industry[['code', 'industry']], how='left', on='code')
    
#     for name in tqdm(['close', 'volume', 'amount', 'turn', 'pctChg', 'pbMRQ']):
#         for day in ['7', '30']:
#             rolling = data.groupby['code'][f'{name}'].rolling(window=int(day), center=False)
#             data[f'{name}_rolling_{day}_mean'] = rolling.mean().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_max'] = rolling.max().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_min'] = rolling.min().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_sum'] = rolling.sum().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_median'] = rolling.median().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_skew'] = rolling.skew().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_kurt'] = rolling.kurt().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_std'] = rolling.std().reset_index(drop=True)
#             data[f'{name}_rolling_{day}_mad'] = rolling.mad()
#             data[f'{name}_rolling_{day}_autocorr1'] = rolling.autocorr(1)
#             data[f'{name}_rolling_{day}_autocorr2'] = rolling.autocorr(2)
    
            
    
    return data.iloc[:train_len].reset_index(drop=True), data.iloc[train_len:].reset_index(drop=True)

In [6]:
train, test = feature_engineer(train, test)

f_train_path = 'stockdata/d_data/f_train_debug.csv'
f_test_path = 'stockdata/d_data/f_test_debug.csv'
train.to_csv(f_train_path, index=False)
test.to_csv(f_test_path, index=False)

In [10]:
train

,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,peTTM,psTTM,pcfNcfTTM,pbMRQ,isST,industry,label
0,2022-01-11,sz.301136,25.00,27.00,23.10,24.00,10.52,42851346.0,1.050813e+09,3,76.1782,1,128.1369,81.088200,10.114170,-335.195141,8.374159,0.0,28,0
1,2022-01-12,sz.301136,22.00,24.10,21.50,22.80,24.00,31008952.0,6.966526e+08,3,55.1256,1,-5.0000,77.033790,9.608462,-318.435384,7.955451,0.0,28,1
2,2022-01-13,sz.301136,22.70,24.18,22.01,22.99,22.80,28491902.0,6.592639e+08,3,50.6510,1,0.8333,77.675738,9.688532,-321.089013,8.021747,0.0,28,0
3,2022-01-14,sz.301136,21.83,23.98,20.75,22.81,22.99,32919762.0,7.260601e+08,3,58.5225,1,-0.7829,77.067577,9.612676,-318.575049,7.958940,0.0,28,1
4,2022-01-17,sz.301136,22.31,25.48,21.90,25.12,22.81,37451805.0,8.997390e+08,3,66.5793,1,10.1271,84.872316,10.586165,-350.837581,8.764953,0.0,28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2022-05-26,sz.301136,21.76,22.40,21.76,22.15,21.76,7269460.0,1.612011e+08,3,12.9232,1,1.7923,78.245600,9.007777,9.852775,4.094802,0.0,28,0
88,2022-05-27,sz.301136,22.15,22.45,21.65,21.85,22.15,6134622.0,1.354502e+08,3,10.9057,1,-1.3544,77.185840,8.885775,9.719328,4.039342,0.0,28,1
89,2022-05-30,sz.301136,21.84,22.21,21.62,22.20,21.85,4958825.0,1.091256e+08,3,8.8155,1,1.6018,78.422227,9.028111,9.875016,4.104046,0.0,28,1
90,2022-05-31,sz.301136,22.20,22.54,21.80,22.27,22.20,5868198.0,1.299427e+08,3,10.4321,1,0.3153,78.669504,9.056578,9.906153,4.116986,0.0,28,0


In [7]:
train['label'] = [0 if x>0 else 1 for x in (train.close - train.shift(-1).close)]
test['label'] = [0 if x>0 else 1 for x in (test.close - test.shift(-1).close)]

ycol = 'label'
feature_names = list(
    filter(lambda x: x not in [ycol, 'code', 'date', ''], train.columns))

# print(feature_names)

In [8]:
params = {'n_estimators':5000,
      'learning_rate': 0.05,
      'max_depth': 7,
      'early_stopping_rounds':1000,
      'loss_function':'Logloss',
      'max_bin':512,
      'subsample':0.8,
      'bootstrap_type':'Poisson',
      'random_seed':np.random.randint(0,2021)}

model = cb.CatBoostClassifier(eval_metric="AUC", task_type='GPU', **params)

X_train = train[feature_names]
Y_train = train[ycol]

X_val = test[feature_names]
Y_val = test[ycol]


lgb_model = model.fit(X_train,
                      Y_train,
                      # eval_names=['train', 'valid'],
                      # eval_set=[(X_train, Y_train), (X_val, Y_val)],
                      eval_set=(X_val, Y_val),
                      verbose=200)


df_importance = pd.DataFrame({
    'column': feature_names,
    'importance': lgb_model.feature_importances_,
})

# lgb_model.save_model(f'cb_{frequency}.model')

0:	learn: 0.7938534	test: 0.5512821	best: 0.5512821 (0)	total: 72.3ms	remaining: 6m 1s
200:	learn: 1.0000000	test: 0.4529915	best: 0.5512821 (0)	total: 6.62s	remaining: 2m 38s
400:	learn: 1.0000000	test: 0.4529915	best: 0.5512821 (0)	total: 13.1s	remaining: 2m 30s
600:	learn: 1.0000000	test: 0.4871795	best: 0.5512821 (0)	total: 19.5s	remaining: 2m 22s
800:	learn: 1.0000000	test: 0.4957265	best: 0.5512821 (0)	total: 26.1s	remaining: 2m 16s
1000:	learn: 1.0000000	test: 0.5042735	best: 0.5512821 (0)	total: 32.6s	remaining: 2m 10s
bestTest = 0.5512820631
bestIteration = 0
Shrink model to first 1 iterations.


In [9]:
lgb_model.save_model(f'cb_f1_debug.model')